# Load the Population Receptive Field properties for the Connective Fields

In [21]:
# Load the necessary libraries 
import pandas as pd
import numpy as np
import os 
import pickle 
from nibabel.freesurfer.io import read_morph_data, write_morph_data

In [22]:
# Define whether you want to run the code on one subject or on the whole dataset

# 1. Run the code on one specific subject 
# subjects = ['sub-02']

# 2. Run the code on glaucoma patients 
subjects = [f'sub-{i:02}'for i in range(45,  47)]

# 3. Run the code on healthy controls 
# subjects = [f'sub-{i:02}'for i in range(21, 47)]

In [23]:
# Define the main variables 
target = 'V2'
source = 'V1'
hemi = 'rh' 
atlas = 'manual'
delineation = 'manualdelin'
denoising = 'nordic'
task = 'RET'
MAIN_PATH = '/Volumes/FedericaCardillo/pre-processing/projects/PROJECT_EGRET-AAA/derivatives'
freesurfer = f"{MAIN_PATH}/freesurfer"

In [24]:
# Set up all the functions needed to extract the pRF mapping properties 
class PRFModel:
    def __init__(self, r2, size, ecc, angle):
        self.r2 = r2        
        self.size = size
        self.ecc = ecc    
        self.angle = angle     

def pickle_file(filepath):
    if not os.path.isfile(filepath):
        raise FileNotFoundError(f"File not found: {filepath}")
    with open(filepath, 'rb') as file: 
        return pickle.load(file)

def load_prf(subj, main_path, atlas, denoising, task):
    filepath = os.path.join(main_path, f'pRFM/{subj}/ses-02/{denoising}/model-{atlas}-nelder-mead-GM_desc-prf_params_{task}.pkl') 
    pkl_data = pickle_file(filepath)                       
    prf_params = pkl_data['model'].iterative_search_params     
    prf_voxels = np.where(pkl_data['rois_mask'] == 1)[0] 
    prf_voxels= prf_voxels
    return prf_params, prf_voxels

def filter_prf(prf_params, prf_voxels):
    return PRFModel(
        r2=prf_params[:, 7],                                              
        size=prf_params[:, 2],                                              
        ecc=np.sqrt(prf_params[:, 1]**2 + prf_params[:, 0]**2),      
        angle=np.arctan2(prf_params[:, 1], prf_params[:, 0]))

def extract_prf(subj, MAIN_PATH, atlas, denoising, task, csv_path):
    best_fit = pd.read_csv(csv_path)
    prf_params, prf_voxels = load_prf(subj, MAIN_PATH, atlas, denoising, task)
    prf_model = filter_prf(prf_params, prf_voxels)
    lh_c = read_morph_data(f'{freesurfer}/{subj}/surf/lh.curv')
    numel_lh = lh_c.shape[0]

    if hemi == 'rh':
        prf_voxels_rh = prf_voxels[prf_voxels >= numel_lh] 
        prf_voxels_rh_adjusted = prf_voxels_rh - numel_lh 
        prf_ecc_rh = prf_model.ecc[prf_voxels >= numel_lh] 
        prf_angle_rh = prf_model.angle[prf_voxels >= numel_lh]
        voxel_to_ecc = dict(zip(prf_voxels_rh_adjusted, prf_ecc_rh))
        voxel_to_angle = dict(zip(prf_voxels_rh_adjusted, prf_angle_rh))
    else: 
        voxel_to_ecc = dict(zip(prf_voxels, prf_model.ecc))
        voxel_to_angle = dict(zip(prf_voxels, prf_model.angle))
        
    best_fit['Source Eccentricity'] = best_fit['Source Vertex Index'].map(voxel_to_ecc)
    best_fit['Source Polar Angle'] = best_fit['Source Vertex Index'].map(voxel_to_angle)

    output = f'/Volumes/FedericaCardillo/results/{subj}/{hemi}/{target}->{source}/best_fits_prf.csv'
    best_fit.to_csv(output, index=False)
    return best_fit

In [25]:
for subj in subjects: 
    
    best_fit_path = f'/Volumes/FedericaCardillo/results/{subj}/{hemi}/{target}->{source}/best_fits.csv' 
    if not os.path.exists(best_fit_path):
        print(f'Best fit file not found for {subj}.')
        continue

    best_fit_prf = extract_prf(subj, MAIN_PATH, atlas, denoising, task, best_fit_path)
    print(best_fit_path)
    target_idx = best_fit_prf['Target Vertex Index'].values.astype(int)
    
    # ECCENTRICITY
    target_ecc = best_fit_prf['Source Eccentricity'].values
    curv = read_morph_data(f'{freesurfer}/{subj}/surf/{hemi}.curv')
    masked_ecc = np.zeros(curv.shape[0])
    masked_ecc[:] = 50 
    masked_ecc[target_idx] = target_ecc
    ecc_output = f'{freesurfer}/{subj}/surf/{hemi}.ecc_{target}{source}'
    write_morph_data(ecc_output, masked_ecc)

    # POLAR ANGLE 
    target_pol = best_fit_prf['Source Polar Angle'].values
    masked_pol = np.zeros(curv.shape[0])
    masked_pol[:] = 50 
    masked_pol[target_idx] = target_pol
    pol_output = f'{freesurfer}/{subj}/surf/{hemi}.pol_{target}{source}'
    write_morph_data(pol_output, masked_pol)

    # VARIANCE EXPLAINED 
    target_ve = best_fit_prf['Best Variance Explained Finer'].values
    masked_ve = np.zeros(curv.shape[0])
    masked_ve[:] = 50 
    masked_ve[target_idx] = target_ve
    ve_output = f'{freesurfer}/{subj}/surf/{hemi}.ve_{target}{source}'
    write_morph_data(ve_output, masked_ve)

    # SIGMA 
    target_sigma = best_fit_prf['Best Sigma Finer'].values
    masked_sigma = np.zeros(curv.shape[0])
    masked_sigma[:] = 50 
    masked_sigma[target_idx] = target_sigma
    sigma_output = f'{freesurfer}/{subj}/surf/{hemi}.sigma_{target}{source}'
    write_morph_data(sigma_output, masked_sigma)

/Volumes/FedericaCardillo/results/sub-45/rh/V2->V1/best_fits.csv
Best fit file not found for sub-46.
